In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tokenization import return_tokens
from julia import Main
import h5py

In [3]:
# https://github.com/EBjerrum/SMILES-enumeration

In [4]:
Main.eval('using JLD; data = load("data.jld")')

{'Y': array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=float32),
 'X': [array([[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
    

In [5]:
data = Main.data

In [6]:
X = np.array(data['X'])
Y = np.array(data['Y'])

In [7]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=.1)
trainX.shape, testX.shape, trainY.shape, testY.shape

((16410, 190, 72), (1824, 190, 72), (16410, 2), (1824, 2))

In [7]:
vocab = pd.read_csv(f'{os.getcwd()}\\vocab.csv')

In [8]:
tokenizer = {i : n for n, i in enumerate(vocab['tokens'].to_list())}

In [9]:
reverse_tokenizer = {value: key for key, value in tokenizer.items()}

In [10]:
convert_back = lambda x: ''.join(reverse_tokenizer.get(np.argmax(i)-1, '') for i in x)

In [11]:
opt = Adam(.0001)
input_shape = trainX[0].shape

In [12]:
model = Sequential()
model.add(LSTM(128,
               input_shape=input_shape,
               dropout=0.2))
model.add(Dense(64, activation="swish"))
model.add(Dense(32, activation="swish"))
model.add(Dense(2, activation="swish"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               102912    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 113,314
Trainable params: 113,314
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
model.fit(trainX, trainY, batch_size=32, epochs=100, validation_data=(testX, testY))

Epoch 1/100
513/513 [==============================] - 99s 184ms/step - loss: 0.6950 - accuracy: 0.4317 - val_loss: 0.6743 - val_accuracy: 0.4150
Epoch 2/100
513/513 [==============================] - 107s 208ms/step - loss: 0.6746 - accuracy: 0.4126 - val_loss: 0.6650 - val_accuracy: 0.3777
Epoch 3/100
513/513 [==============================] - 113s 220ms/step - loss: 0.6570 - accuracy: 0.3862 - val_loss: 0.6369 - val_accuracy: 0.3448
Epoch 4/100
513/513 [==============================] - 114s 223ms/step - loss: 0.6492 - accuracy: 0.3707 - val_loss: 0.6283 - val_accuracy: 0.3432
Epoch 5/100
212/513 [===========>..................] - ETA: 1:09 - loss: 0.6481 - accuracy: 0.3734